In [ ]:
from os import environ

import neptune

from core_analysis.utils.constants import NEPTUNE_PROJECT, NEPTUNE_API_TOKEN

environ["NEPTUNE_PROJECT"] = NEPTUNE_PROJECT
environ["NEPTUNE_API_TOKEN"] = NEPTUNE_API_TOKEN

run = neptune.init_run(name="Test HIPIE")

In [ ]:
import os

import torch
from PIL import Image
from matplotlib import pyplot as plt

from detectron2.utils.visualizer import Visualizer
from projects.HIPIE.demo_lib.part_segm_demo import PartSegmDemo
from detectron2.data.detection_utils import convert_PIL_to_numpy

In [ ]:
run["checkpoint"] = ckpt = "weights/vit_h_cloud.pth"
img_path = R"C:\Users\Travail\Desktop\Postdoctorat\Erwan\core-analysis\images\OS-13-217\OS-13-217 Bx 61-63 230.05-241.71m.JPG"
# img_path = R"C:\Users\Travail\Desktop\Postdoctorat\Erwan\dorchester.jpg"
run["image_path"] = img_path
run["use_default_labels"] = use_default_labels = False
run["instance_thres"] = instance_thres = 0.2

# Load Models

In [ ]:
current_folder = globals()["_dh"][0]
print(current_folder)

os.chdir(os.path.join(current_folder, ".."))

config_file = "projects/HIPIE/configs/image_joint_vit_huge_32g_pan_maskdino_parts.yaml"

uninext_demo = PartSegmDemo(config_file=config_file, weight=ckpt, device="cuda")
uninext_demo.demo.predictor.model.device = "cuda"

# Set up an image

In [ ]:
# url = 'https://raw.githubusercontent.com/CompVis/latent-diffusion/main/data/inpainting_examples/overture-creations-5sI6fQgYIuo.png'
# url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg/405px-Mona_Lisa%2C_by_Leonardo_da_Vinci%2C_from_C2RMF_retouched.jpg'
# url = 'https://encrypted-tbn0.gstatic.com/licensed-image?q=tbn:ANd9GcSh82Dm7OEK7SU7Rnv3Fa-9bi6BZrMM0NJvtm94eGajmxl7mObM7Jp9h3z5UfxaY5IsXGBoB9IX1QMVMlE'
# url = 'https://hips.hearstapps.com/hmg-prod/images/dog-puppy-on-garden-royalty-free-image-1586966191.jpg?crop=0.752xw:1.00xh;0.175xw,0&resize=1200:*'
# image = Image.open(requests.get(url,stream=True).raw)
# image = Image.open("assets/demo_hipie.jpg")
image = Image.open(img_path)
# run["i_min"], run["i_max"] = i_min, i_max = 200 * 7 + 105, 250 * 7 - 70
# run["j_min"], run["j_max"] = j_min, j_max = 230 * 7 + 80 * 2, 280 * 7
run["i_min"], run["i_max"] = i_min, i_max = 200 * 7 + 120, 250 * 7 - 50
run["j_min"], run["j_max"] = j_min, j_max = 400 * 7 + 40, 440 * 7 - 40
# run["i_min"], run["i_max"] = i_min, i_max = 0, image.height
# run["j_min"], run["j_max"] = j_min, j_max = 0, image.width
image = image.crop((j_min, i_min, j_max, i_max))
run["rotation"] = rotation = 90
image = image.rotate(rotation, expand=1)
image_np_d2 = convert_PIL_to_numpy(image, format="BGR")
run["undersampling"] = undersampling = 1
image_np_d2 = image_np_d2[::undersampling, ::undersampling]
run["image_height"] = image.height
run["image_width"] = image.width
run["input_image"].upload(neptune.types.File.as_image(image_np_d2[..., ::-1] / 255))
plt.imshow(image_np_d2[..., ::-1])

In [ ]:
from detectron2.projects.hipie.data.coco_dataset_mapper_uni import get_openseg_labels

COCO_OPENSEG_LABELS = get_openseg_labels("coco_panoptic")
coco_labels = dict(
    things_labels=[x["name"] for x in get_openseg_labels("coco_panoptic")[:80]],
    stuff_labels=[x["name"] for x in get_openseg_labels("coco_panoptic")[80:]],
)
custom_labels = dict(
    things_labels=["quartz", "fracture"],
    stuff_labels=["core"],
)

if use_default_labels:
    selected_labels = coco_labels
else:
    selected_labels = custom_labels
run["labels"] = {
    key: neptune.types.StringSeries(value) for key, value in selected_labels.items()
}

In [ ]:
mask = uninext_demo.foward_panoptic(
    image_np_d2, do_part=True, instance_thres=instance_thres, **selected_labels
)

# Panoptic Seg

In [ ]:
visualizer = Visualizer(image_np_d2[..., ::-1], metadata=mask["meta_data"])
visualizer.draw_panoptic_seg(mask["panoptic_seg"][0], mask["panoptic_seg"][1])
fig = visualizer.get_output().fig
display(fig)
run["panoptic_seg"].upload(fig)

#  Part Seg

In [ ]:
visualizer = Visualizer(image_np_d2[..., ::-1])
visualizer.overlay_instances(
    masks=torch.stack(mask["output_refined"][0]), labels=mask["output_refined"][1]
)
fig = visualizer.get_output().fig
display(fig)
run["part_seg"].upload(fig)

# Instance Seg

In [ ]:
visualizer = Visualizer(image_np_d2[..., ::-1], metadata=mask["meta_data"])
visualizer.draw_instance_predictions(mask["instances"])
fig = visualizer.get_output().fig
display(fig)
run["instance_seg"].upload(fig)

In [ ]:
ref_str = "a core segment"
mask = uninext_demo.foward_reference(image_np_d2, ref_str, "")

In [ ]:
vis = Visualizer(image_np_d2[..., ::-1])
# vis.overlay_instances(masks=mask['instance_mask'])
# vis.overlay_instances(masks=part_mask)
vis.overlay_instances(masks=[mask["final_mask"][0].numpy()], labels=[ref_str])
vis.get_output().fig

In [ ]:
run.stop()